J'essaie de scraper des infos sur la page de présentation Fragantica de Black Phantom : https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html

Je veux scraper ces infos :
Nom du Parfum
Nom de la marque
Launch year
Perfume rating
Nombre de votant sur le Perfume rating
Saisonnalité (Hiver, automne… peut etre gardé juste la + votée)
Parfumeur
Main accord
‘Perfume Pyramide’  = [‘Top note’ , ‘Middle note’, Base note’]
Genre (en prenant celui qui à le plus de vote)
Longevite, Sillage (en prenant les votes)


## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import cloudscraper

import numpy as np
import pandas as pd


## Les fonctions

### DESROMAIS INUTILE fonction pour récuperer le code html d'une page parfum

In [2]:
#Récupère le HTML de la page correspondant à l'URL donnée.
def get_html_headless(url):
    """
    Ouvre Chrome en mode headless, va sur l'URL, 
    attend x seconde, récupère le HTML.
    """

    #1) ✅ Configuration Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Désactiver l'ouverture de Chrome
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--blink-settings=imagesEnabled=false") # Pas besoin d'images = gain de temps
    options.add_argument("--headless=new")  # Nouvelle version plus rapide du headless
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

    # 2) Création du driver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options)
        
    # 1) Accreder à l'URL
    driver.get(url)
    
    # 3) Récupère le code HTML
    html_content = driver.page_source

    driver.quit()

    return html_content

### Fonctions intermédiaire pour récuperer les infos sur un parfum

In [3]:
# 1) Nom du parfum
def extract_perfume_name(html_content):
    """Extrait le nom du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', html_content)
    return match.group(1).replace('-', ' ') if match else None

# 2) Marque
def extract_brand_name(html_content):
    """Extrait la marque du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/([^/]+)/', html_content)
    return match.group(1).replace('-', ' ') if match else None

#3) Parfumeur
def extract_nose(soup):
    """Extrait le nom du parfumeur"""
    nose_el = soup.select_one('div.cell a[href^="/noses/"]')
    return nose_el.get_text(strip=True) if nose_el else None

#4) Année de sortie
def extract_launch_year(soup):
    """Extrait l'année de sortie du parfum"""
    title_el = soup.find('title')
    if title_el:
        possible_year = title_el.get_text(strip=True)[-4:]
        if possible_year.isdigit():
            return possible_year
    return None

#5) Perfume rating
def extract_rating(soup):
    """Extrait la note du parfum"""
    rating_el = soup.select_one('span[itemprop="ratingValue"]')
    return rating_el.get_text(strip=True) if rating_el else None

#6) Nombre de votes
def extract_rating_count(soup):
    """Extrait le nombre de votes"""
    rating_count_el = soup.select_one('span[itemprop="ratingCount"]')
    return rating_count_el.get_text(strip=True) if rating_count_el else None

#7) Accords principaux
def extract_main_accords(soup):
    """Extrait les accords principaux du parfum"""
    main_accords_el = soup.find_all('div', class_='cell accord-box')
    return [element.get_text(strip=True) for element in main_accords_el if element.get_text(strip=True)]

#8) Genre
def extract_gender(gender_list):
    """Détermine le genre du parfum en fonction des votes"""
    if not gender_list:
        return None
    
    sum_votes = np.sum([int(element[1]) for element in gender_list])
    
    if sum_votes > 8:
        female_count = int(gender_list[0][1]) + int(gender_list[1][1])
        male_count = int(gender_list[3][1]) + int(gender_list[4][1])
        unisex_count = 1.2 * int(gender_list[2][1])

        if female_count > male_count and female_count > unisex_count:
            return "female"
        elif male_count > unisex_count:
            return "male"
        else:
            return "unisex"
    return None

#9) Longévité
def extract_longevity(longevity_list):
    """Détermine la longévité du parfum"""
    if not longevity_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in longevity_list])
    if sum_votes > 8:
        longevity = np.argmax([int(element[1]) for element in longevity_list])
        return longevity_list[longevity][0]
    return None

#10) Sillage
def extract_sillage(sillage_list):
    """Détermine le sillage du parfum"""
    if not sillage_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in sillage_list])
    if sum_votes > 8:
        sillage = np.argmax([int(element[1]) for element in sillage_list])
        return sillage_list[sillage][0]
    return None

#11) Ressenti du prix
def extract_price_feeling(price_feeling_list):
    """Détermine le ressenti du prix du parfum"""
    if not price_feeling_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in price_feeling_list])
    if sum_votes > 8:
        price_feeling = np.argmax([int(element[1]) for element in price_feeling_list])
        return price_feeling_list[price_feeling][0]
    return None

#12) Pyramide olfactive
def extract_pyramid_ingredients(soup, pyramid_section):
    """Extrait les ingrédients d'une section de la pyramide olfactive"""
    pyramid_ingredients = []
    #On repère la bonne zone grâce à la section de la pyramide
    header = soup.find('h4', string=lambda text: text and pyramid_section in text)
    if header:
        #Les noms des ingrédients sont stockés dans la div suivante
        div = header.find_next('div')
        if div:
            ingredients_divs = div.find_all('div')
            for ingredient_div in ingredients_divs:
                a_tag = ingredient_div.find('a')
                if a_tag and a_tag.next_sibling:
                    ingredient_name = a_tag.next_sibling.strip()
                    if ingredient_name not in pyramid_ingredients:
                        pyramid_ingredients.append(ingredient_name)
    return pyramid_ingredients

### Fonctions intermediaire pour le tableau avec Gender, Sillage, loongevity et Price feeling

In [4]:
def extract_votes(soup):
    """Extrait les catégories et leurs votes à partir du HTML."""
    tableau_votes = []
    tab_rows = soup.select('div.grid-x.grid-margin-x')  # Sélection des blocs contenant les votes
    
    for row in tab_rows:
        category_el = row.select_one('span.vote-button-name')  # Nom de la catégorie
        votes_el = row.select_one('span.vote-button-legend')  # Nombre de votes

        if category_el and votes_el:
            category = category_el.get_text(strip=True)
            votes = votes_el.get_text(strip=True)

            # Vérifier si le nombre de votes est bien un chiffre
            if votes.isdigit():
                tableau_votes.append((category, votes))

    return tableau_votes

def filter_votes(tableau_votes, labels, remove_first=False, remove_last=False):
    """
    Filtre les votes selon un ensemble de labels et enlève potentiellement le premier ou dernier élément.
    
    :param tableau_votes: Liste des votes [(catégorie, nombre)]
    :param labels: Ensemble des labels à filtrer
    :param remove_first: Supprime le premier élément si True
    :param remove_last: Supprime le dernier élément si True
    :return: Liste filtrée et éventuellement modifiée
    """
    filtered_list = [element for element in tableau_votes if element[0] in labels]

    if remove_first and filtered_list:
        filtered_list.pop(0)
    if remove_last and filtered_list:
        filtered_list.pop()
    
    return filtered_list

### Scrape perfume info

In [35]:
def scrape_perfume_info(html_content, soup):
    """
    Scrap les infos principales depuis la page Fragrantica d'un parfum.
    Retourne un dictionnaire avec ces informations.
    """
    
    
    perfume_data = {
        "nom_parfum": extract_perfume_name(html_content),
        "marque": extract_brand_name(html_content),
        "nose": extract_nose(soup),
        "launch_year": extract_launch_year(soup),
        "rating_value": extract_rating(soup),
        "rating_count": extract_rating_count(soup),
    }

    # 3) Retourner le dictionnaire
    return perfume_data

## Le script

### Les options pour webdriver

### Un script utilisant les fonctions

### Un script sans les fonctions

- nom_parfum
      - marque
      - launch_year
      - perfume_rating
      - nb_votes
      - parfumeur
      - main_accord
      - perfume_pyramid (dict avec top/middle/base)
      - genre
      - longevite_votes
      - sillage_votes


In [6]:
data = {}

# 1) Nom du parfum        
match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', html_content)
if match:
    perfume_name = match.group(1)
    perfume_name = perfume_name.replace('-', ' ')
    data['nom_parfum'] = perfume_name
else:
    data['nom_parfum'] = None

#2) Marque
match = re.search(r'/perfume/([^/]+)/', html_content)
if match:
    brand_name = match.group(1)
    brand_name = brand_name.replace('-', ' ')
    data['marque'] = brand_name
else:
    data['marque'] = None

#3) Parfumeur
data['nose'] = None
nose_el = soup.select_one('div.cell a[href^="/noses/"]')
if nose_el:
    data['nose'] = nose_el.get_text(strip=True)

#4) Année de sortie
data['launch_year'] = None
title_el = soup.find('title')
if title_el:
    possible_year = title_el.get_text(strip=True)[-4:] #Les 4 derniers caractères du titre
    if possible_year.isdigit():
        data['launch_year'] = possible_year

#5) Perfume rating (note)
## ATTENTION, valeur en string, pas en float
data['rating_value'] = None
rating_el = soup.select_one('span[itemprop="ratingValue"]').get_text(strip=True)
if rating_el:
    data['rating_value'] = rating_el  

#6) Nb de votes
data['rating_count'] = None
rating_count_el = soup.select_one('span[itemprop="ratingCount"]').get_text(strip=True)
if rating_count_el:
    data['rating_count'] = rating_count_el


#7)Main accord
data['main_accords'] = []
main_accords_el = soup.find_all('div', class_= 'cell accord-box')
for element in main_accords_el:
    accord = element.get_text(strip=True)
    if accord:
        data['main_accords'].append(accord)

#8) Gender
##Regle de calcul : max ( female + more female , male + more male , 1,2* unisex)
data['gender'] = None
sum_votes = np.sum([int(element[1]) for element in gender_list])

if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None

    female_count = int(gender_list[0][1]) + int(gender_list[1][1])
    male_count = int(gender_list[3][1]) + int(gender_list[4][1])
    unisex_count = 1.2 * int(gender_list[2][1])
    
    if female_count > male_count and female_count > unisex_count:
        data['gender'] = "female"
    else:
        if male_count > unisex_count:
            data['gender'] = "male"
        else :
            data['gender'] = "unisex"

#9) Longévité
data['longevity'] = None
sum_votes = np.sum([int(element[1]) for element in longevity_list])
if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None
    longevity = np.argmax([int(element[1]) for element in longevity_list])
    data['longevity'] = longevity_list[longevity][0]

#10) Sillage
data['sillage'] = None
sum_votes = np.sum([int(element[1]) for element in sillage_list])
if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None
    sillage = np.argmax([int(element[1]) for element in sillage_list])
    data['sillage'] = sillage_list[sillage][0]
    
#11) Price feeling
data['price_feeling'] = None
sum_votes = np.sum([int(element[1]) for element in price_feeling_list])
if sum_votes > 8 : #Si il n'y a pas assez de votes on reste à None
    price_feeling = np.argmax([int(element[1]) for element in price_feeling_list])
    data['price_feeling'] = price_feeling_list[price_feeling][0]

print(data)


NameError: name 'html_content' is not defined

In [ ]:
#Les listes des votes pour Longevity, Sillage, Gender, Price Feeling
longevity_list = []
sillage_list = []
gender_list = []
price_feeling_list = []
#Sélecteur des blocs qui contiennent la catégorie + nombre de votes
tab_rows = soup.select('div.grid-x.grid-margin-x')  #On selectionne trop de chose donc on va devoir filtrer
tableau_votes = []
for row in tab_rows:
    # Récupérer le nom de la catégorie (ex: "female") dans <span class="vote-button-name">
    category_el = row.select_one('span.vote-button-name')
    # Récupérer le nombre de votes dans <span class="vote-button-legend">
    votes_el = row.select_one('span.vote-button-legend')
    if category_el and votes_el:
        category = category_el.get_text(strip=True)
        votes = votes_el.get_text(strip=True)   
        tableau_votes.append((category, votes))

for element in tableau_votes:
    if not element[1].isdigit(): #Si le nombre de votes n'est pas un nombre, et oui ça arrive !
        tableau_votes.remove(element)

# Définition des groupes
longevity_labels = {"very weak", "weak", "moderate", "long lasting", "eternal"}
sillage_labels = {"intimate", "moderate", "strong", "enormous"}
gender_labels = {"female", "more female", "unisex", "more male", "male"}
price_feeling_labels = {"way overpriced", "overpriced", "ok", "good value", "great value"}
# Séparation des données
longevity_list = [element for element in tableau_votes if element[0] in longevity_labels]
longevity_list.pop() #On enlève le dernier élément qui est le modérate sillage
sillage_list = [element for element in tableau_votes if element[0] in sillage_labels]
sillage_list.pop(0) #On enlève le premier élément qui est le modérate longevity
gender_list = [element for element in tableau_votes if element[0] in  gender_labels]
price_feeling_list = [element for element in tableau_votes if element[0] in  price_feeling_labels]




[('very weak', '113'), ('weak', '342'), ('moderate', '2458'), ('long lasting', '4827'), ('eternal', '1399')]
[('intimate', '421'), ('moderate', '4052'), ('strong', '3606'), ('enormous', '798')]


## Brouillon

In [29]:
url = "https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html"

#✅ Configuration Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Désactiver l'ouverture de Chrome
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--blink-settings=imagesEnabled=false") # Pas besoin d'images = gain de temps
options.add_argument("--headless=new")  # Nouvelle version plus rapide du headless
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# 2) Création du driver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options)

# 3) Récupération du contenu HTML
driver.get(url)
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
driver.quit()

In [30]:
def extract_gender(votes_dict):
    """Détermine le genre du parfum en fonction des votes extraits."""
    gender_labels = ["female", "more female", "unisex", "more male", "male"]
    gender_votes = [votes_dict.get(label, 0) for label in gender_labels]
    
    if sum(gender_votes) <= 8:
        return None

    female_count = gender_votes[0] + gender_votes[1]
    male_count = gender_votes[3] + gender_votes[4]
    unisex_count = 1.2 * gender_votes[2]

    return max(("female", female_count),("male", male_count),("unisex", unisex_count),key=lambda x: x[1])[0]

def extract_longevity(votes_dict):
    """Détermine la longévité dominante en fonction des votes."""
    longevity_labels = ["very weak", "weak", "moderate", "long lasting", "eternal"]
    longevity_votes = [votes_dict.get(label, 0) for label in longevity_labels]

    if sum(longevity_votes) > 8:
        return longevity_labels[np.argmax(longevity_votes)]

    return None

def extract_sillage(votes_dict):
    """Détermine le sillage dominant en fonction des votes."""
    sillage_labels = ["intimate", "average", "strong", "enormous"]
    sillage_votes = [votes_dict.get(label, 0) for label in sillage_labels]

    if sum(sillage_votes) > 8:
        return sillage_labels[np.argmax(sillage_votes)]

    return None

def extract_price_feeling(votes_dict):
    """Détermine la perception du prix en fonction des votes."""
    price_labels = ["way overpriced", "overpriced", "ok", "good value", "great value"]
    price_votes = [votes_dict.get(label, 0) for label in price_labels]

    if sum(price_votes) > 8:
        return price_labels[np.argmax(price_votes)]

    return None


In [33]:
def extract_all_votes(soup):
    """
    Extrait tous les votes de la page sous forme d'un dictionnaire.
    
    :param soup: Objet BeautifulSoup contenant le HTML.
    :return: Dictionnaire {'category': votes}
    """
    votes_dict = {}
    seen_moderate = 0

    # Sélectionner tous les blocs contenant les votes
    vote_rows = soup.select('div.grid-x.grid-margin-x')

    for row in vote_rows:
        category_el = row.select_one('span.vote-button-name')
        votes_el = row.select_one('span.vote-button-legend')

        if category_el and votes_el:
            category = category_el.get_text(strip=True)
            votes = votes_el.get_text(strip=True)

            if category == "moderate":
                    seen_moderate += 1
                    if seen_moderate == 2:
                        category = "average"

            # Vérifier si le vote est bien un nombre    
            if votes.isdigit():
                votes_dict[category] = int(votes)
            

    return votes_dict


In [34]:
BP_votes = extract_all_votes(soup)
print(BP_votes)
print(extract_gender(BP_votes))
print(extract_longevity(BP_votes))
print(extract_sillage(BP_votes))    
print(extract_price_feeling(BP_votes))


{'very weak': 128, 'weak': 226, 'moderate': 1084, 'long lasting': 1847, 'eternal': 989, 'intimate': 454, 'average': 1943, 'strong': 1503, 'enormous': 788, 'female': 175, 'more female': 258, 'unisex': 2030, 'more male': 1050, 'male': 455, 'way overpriced': 904, 'overpriced': 1369, 'ok': 879, 'good value': 183, 'great value': 62}
unisex
long lasting
average
overpriced
